In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine-date')

In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

In [4]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state = 42)

In [5]:
print(sub_input.shape, val_input.shape, train_input.shape)

(4157, 3) (1040, 3) (5197, 3)


In [7]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [8]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)    # 평가할 모델 객체, 훈련 세트 input, 훈련 세트 target

In [9]:
print(scores)

{'fit_time': array([0.01852393, 0.01362491, 0.01087332, 0.0111177 , 0.01036191]), 'score_time': array([0.00173736, 0.00129962, 0.00130796, 0.00127363, 0.00126767]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [10]:
import numpy as np
print(np.mean(scores['test_score']))    # 이름은 test_score지만 검증 폴드의 점수, 혼동 X

0.855300214703487


In [13]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [14]:
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)    # 세트를 섞고 10-폴드 교차 검증
scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [15]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}    # 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 생성

In [18]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs = -1)

In [19]:
gs.fit(train_input, train_target)    # 하이퍼 파라미터 5개, 5-fold cv 이므로 25개 모델 생성

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [20]:
dt = gs.best_estimator_    # 25개 중 가장 성능 좋은 모델 반환

In [21]:
print(dt.score(train_input, train_target))

0.9615162593804117


In [24]:
gs.best_params_    # 최적의 매개변수 반환

{'min_impurity_decrease': 0.0001}

In [25]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [26]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [27]:
# 결정트리 다양한 하이퍼파라미터
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
}

In [28]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs = -1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [29]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [30]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [32]:
from scipy.stats import uniform, randint

In [36]:
rgen = randint(0, 10)
rgen.rvs(10)

array([7, 5, 3, 1, 6, 4, 0, 0, 4, 0])

In [37]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 98, 105, 104, 123,  91, 107,  93,  99,  92,  88]))

In [38]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.66608423, 0.2421435 , 0.66828736, 0.09748424, 0.42198826,
       0.92431017, 0.85721892, 0.67469647, 0.44912161, 0.45288739])

In [41]:
# uniform과 randint 통해 알아서 샘플링

params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25)
}

In [42]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa8829bdc10>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa88294f050>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa8836df2d0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa8829bd710>},
                   random_state=42)

In [43]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [47]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [48]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
